In [1]:
%matplotlib inline
import pandas as pd
import librosa as lib
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Audio
import scipy

In [2]:
#Import music dataframe
musicr = pd.read_pickle('DFs/musicr.pkl')

In [3]:
#import motion dataframe 
motion_all = pd.read_pickle('DFs/motion_all.pkl')

In [4]:
#import demographic information dataframe, includes all participants, but we will only use particpant 1 info for now
demos = pd.read_pickle('DFs/demos_EDA.pkl')

In [5]:
demos.loc[demos['Sex'].isnull(), 'Sex']=1

Reminder of what each look like

In [6]:
musicr.head()

,C,C_sharp,D,D_sharp,E,F,F_sharp,G,G_sharp,A,A_sharp,B
0,0.391866,0.271475,0.543743,0.414470,0.615711,0.905505,0.693868,1.0,0.873349,0.428765,0.196232,0.178688
1,0.391006,0.270759,0.542460,0.419167,0.611465,0.895052,0.696120,1.0,0.865096,0.424725,0.202718,0.177431
2,0.384290,0.269616,0.530845,0.424728,0.596871,0.879191,0.701280,1.0,0.854621,0.410783,0.205452,0.178596
3,0.368410,0.266402,0.511752,0.433351,0.582881,0.878751,0.706602,1.0,0.850467,0.397394,0.211017,0.174648
4,0.356740,0.260931,0.492235,0.435323,0.570510,0.877007,0.707534,1.0,0.849687,0.381047,0.214705,0.177209


In [7]:
motion_all.head()

,index,X,Y,Z,PID,X_disp,Y_disp,Z_disp,X_disp_total,Y_disp_total,...,"(_0.6sec_hist_std_,)","(_0.8sec_hist_std_,)","(_1.0sec_hist_std_,)","(_1.2sec_hist_std_,)","(_1.4sec_hist_std_,)","(_1.6sec_hist_std_,)","(_1.8sec_hist_std_,)","(_2.0sec_hist_std_,)",millisecond,segment
0,1900-01-01 00:00:00.000,159.6240,1845.0448,1732.1037,1,0.1470,0.6545,0.0167,0.1470,0.6545,...,0.402534,0.402534,0.402534,0.402534,0.402534,0.402534,0.402534,0.402534,0,silence
1,1900-01-01 00:00:00.100,159.7710,1844.3903,1732.0870,1,0.1470,0.6545,0.0167,0.1470,0.6545,...,0.402534,0.402534,0.402534,0.402534,0.402534,0.402534,0.402534,0.402534,100,silence
2,1900-01-01 00:00:00.200,160.0116,1844.3733,1732.0775,1,0.2406,0.0170,0.0095,0.3876,0.6715,...,0.402534,0.402534,0.402534,0.402534,0.402534,0.402534,0.402534,0.402534,200,silence
3,1900-01-01 00:00:00.300,160.4118,1844.8992,1732.0579,1,0.4002,0.5259,0.0196,0.7878,1.1974,...,0.402534,0.402534,0.402534,0.402534,0.402534,0.402534,0.402534,0.402534,300,silence
4,1900-01-01 00:00:00.400,160.4588,1846.0076,1732.0856,1,0.0470,1.1084,0.0277,0.8348,2.3058,...,0.402534,0.402534,0.402534,0.402534,0.402534,0.402534,0.402534,0.402534,400,silence


In [8]:
motion_all = motion_all.sort_values(by='index')
motion_all

,index,X,Y,Z,PID,X_disp,Y_disp,Z_disp,X_disp_total,Y_disp_total,...,"(_0.6sec_hist_std_,)","(_0.8sec_hist_std_,)","(_1.0sec_hist_std_,)","(_1.2sec_hist_std_,)","(_1.4sec_hist_std_,)","(_1.6sec_hist_std_,)","(_1.8sec_hist_std_,)","(_2.0sec_hist_std_,)",millisecond,segment
0,1900-01-01 00:00:00,159.6240,1845.0448,1732.1037,1,0.1470,0.6545,0.0167,0.1470,0.6545,...,0.402534,0.402534,0.402534,0.402534,0.402534,0.402534,0.402534,0.402534,0,silence
0,1900-01-01 00:00:00,2510.0008,1124.2335,1708.6818,71,0.1749,0.2873,0.0745,0.1749,0.2873,...,0.309662,0.309662,0.309662,0.309662,0.309662,0.309662,0.309662,0.309662,0,silence
0,1900-01-01 00:00:00,39.2135,280.7011,1789.3417,26,0.4696,0.3262,0.0304,0.4696,0.3262,...,0.193286,0.193286,0.193286,0.193286,0.193286,0.193286,0.193286,0.193286,0,silence
0,1900-01-01 00:00:00,1719.4441,1109.8740,1857.6997,70,0.4568,0.5815,0.0223,0.4568,0.5815,...,0.278487,0.278487,0.278487,0.278487,0.278487,0.278487,0.278487,0.278487,0,silence
0,1900-01-01 00:00:00,42.3865,1159.3252,1884.5194,43,0.3322,0.3693,0.0324,0.3322,0.3693,...,0.193560,0.193560,0.193560,0.193560,0.193560,0.193560,0.193560,0.193560,0,silence
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3560,1900-01-01 00:05:56,1752.0740,1816.8560,1619.4980,20,0.2295,0.8984,0.1044,700.3593,1524.0675,...,0.239857,0.241276,0.273957,0.279096,0.320566,0.162036,0.261072,0.257630,356000,edm
3560,1900-01-01 00:05:56,2480.1530,82.6900,1725.8450,8,0.1089,0.0425,0.0611,834.5131,1939.9632,...,0.619701,0.696003,0.677085,0.690428,0.534325,0.533442,0.353229,0.353102,356000,edm
3560,1900-01-01 00:05:56,158.6100,1854.9490,1715.6250,1,0.1017,0.0044,0.0551,913.0808,2485.3534,...,0.347882,0.287895,0.274467,0.275991,0.174467,0.176022,0.313231,0.341873,356000,edm
3560,1900-01-01 00:05:56,858.1100,210.8100,1573.1610,27,0.0891,0.5384,0.0023,1103.5052,2108.6366,...,0.170198,0.216290,0.192363,0.188474,0.236382,0.244705,0.257317,0.499879,356000,edm


In [9]:
demos.head()

,PID,SID,Age,Sex,Height,Listen,Produce,Dance,Exercise,Tiresome,age_bin,height_bin
0,1,A1,23,0.0,1.72,5,30.0,1.0,7.0,3,3,2
1,2,A2,24,0.0,1.67,10,10.0,2.0,10.0,1,3,2
2,3,A3,27,1.0,1.63,14,1.0,4.0,3.0,4,4,1
3,4,A4,27,0.0,1.75,5,20.0,2.0,2.0,4,4,3
4,5,A9,24,1.0,1.64,15,0.0,2.0,6.0,2,3,1


### Matching Times Between DataFrames
We need to match the musics and motion DataFrames together. We will create a 'time' column that represents milliseconds as we already have for the motion data. The music starts at 3 minutes or 180000 milliseconds.

In [10]:
musicr['millisecond'] = list(range(180000,356010, 10))

In [11]:
musicr.tail()

,C,C_sharp,D,D_sharp,E,F,F_sharp,G,G_sharp,A,A_sharp,B,millisecond
17596,0.551258,0.239016,0.590822,0.848327,0.556109,0.694828,1.0,0.846755,0.592610,0.795401,0.520809,0.609293,355960
17597,0.486828,0.255034,0.616595,0.832311,0.567620,0.685870,1.0,0.849619,0.508727,0.755731,0.539001,0.600682,355970
17598,0.418250,0.280371,0.653649,0.814706,0.601733,0.692381,1.0,0.866687,0.420425,0.728755,0.549097,0.572606,355980
17599,0.418250,0.280371,0.653649,0.814706,0.601733,0.692381,1.0,0.866687,0.420425,0.728755,0.549097,0.572606,355990
17600,0.370047,0.297515,0.677903,0.787486,0.621999,0.695762,1.0,0.880282,0.364317,0.719848,0.556928,0.527846,356000


# Feature Creation
We are going to create some additonal features from the note amplitudes including rolling averages, mins, maxs and rates of change

In [12]:
def create_lags(lags, note_df):
    for note in note_df.columns:
        if note != 'millisecond':
            for rows in lags:
                lag = note_df[note].shift(periods=rows) #5 rows equals 1 second
                lag.rename(note+'_{sec}sec_hist'.format(sec=int(rows/5)), inplace=True)
                note_df = pd.concat([note_df, lag], axis=1)
    return note_df

In [13]:
def aggregate_lags(aggs, history):
    history_aggregation = history.copy()
    for func in aggs:
        for col in history.columns:
            if col != 'millisecond':
                history_aggregation[col+'_{func}'.format(func=func)] = history[col].diff().rolling(5).agg(func)
     
    return history_aggregation

In [14]:
lags= range(5, 55, 5)
hist = create_lags(lags, musicr)
hist   

,C,C_sharp,D,D_sharp,E,F,F_sharp,G,G_sharp,A,...,B_1sec_hist,B_2sec_hist,B_3sec_hist,B_4sec_hist,B_5sec_hist,B_6sec_hist,B_7sec_hist,B_8sec_hist,B_9sec_hist,B_10sec_hist
0,0.391866,0.271475,0.543743,0.414470,0.615711,0.905505,0.693868,1.000000,0.873349,0.428765,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.391006,0.270759,0.542460,0.419167,0.611465,0.895052,0.696120,1.000000,0.865096,0.424725,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.384290,0.269616,0.530845,0.424728,0.596871,0.879191,0.701280,1.000000,0.854621,0.410783,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.368410,0.266402,0.511752,0.433351,0.582881,0.878751,0.706602,1.000000,0.850467,0.397394,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.356740,0.260931,0.492235,0.435323,0.570510,0.877007,0.707534,1.000000,0.849687,0.381047,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17596,0.551258,0.239016,0.590822,0.848327,0.556109,0.694828,1.000000,0.846755,0.592610,0.795401,...,0.689397,0.628860,0.573706,0.564126,0.621190,0.715490,0.825529,0.745469,0.742854,0.615103
17597,0.486828,0.255034,0.616595,0.832311,0.567620,0.685870,1.000000,0.849619,0.508727,0.755731,...,0.689397,0.637443,0.584429,0.564126,0.600969,0.681154,0.822908,0.738143,0.761616,0.650937
17598,0.418250,0.280371,0.653649,0.814706,0.601733,0.692381,1.000000,0.866687,0.420425,0.728755,...,0.658993,0.646374,0.599206,0.565270,0.578153,0.662805,0.822908,0.735880,0.780407,0.650937
17599,0.418250,0.280371,0.653649,0.814706,0.601733,0.692381,1.000000,0.866687,0.420425,0.728755,...,0.621587,0.663144,0.615284,0.561653,0.561970,0.646040,0.780619,0.735818,0.761296,0.686015


In [17]:
agg_funcs = ['mean', 'max', 'min', 'std']
hist_agg = aggregate_lags(agg_funcs, hist)

musicr = pd.concat([musicr, hist_agg])

musicr.bfill(inplace=True)
musicr.ffill(inplace=True)
musicr

,C,C_sharp,D,D_sharp,E,F,F_sharp,G,G_sharp,A,...,B_1sec_hist_std,B_2sec_hist_std,B_3sec_hist_std,B_4sec_hist_std,B_5sec_hist_std,B_6sec_hist_std,B_7sec_hist_std,B_8sec_hist_std,B_9sec_hist_std,B_10sec_hist_std
0,0.391866,0.271475,0.543743,0.414470,0.615711,0.905505,0.693868,1.000000,0.873349,0.428765,...,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722
1,0.391006,0.270759,0.542460,0.419167,0.611465,0.895052,0.696120,1.000000,0.865096,0.424725,...,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722
2,0.384290,0.269616,0.530845,0.424728,0.596871,0.879191,0.701280,1.000000,0.854621,0.410783,...,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722
3,0.368410,0.266402,0.511752,0.433351,0.582881,0.878751,0.706602,1.000000,0.850467,0.397394,...,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722
4,0.356740,0.260931,0.492235,0.435323,0.570510,0.877007,0.707534,1.000000,0.849687,0.381047,...,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17596,0.551258,0.239016,0.590822,0.848327,0.556109,0.694828,1.000000,0.846755,0.592610,0.795401,...,0.010291,0.006476,0.004831,0.011797,0.012607,0.021592,0.026881,0.018162,0.016367,0.008562
17597,0.486828,0.255034,0.616595,0.832311,0.567620,0.685870,1.000000,0.849619,0.508727,0.755731,...,0.011651,0.006571,0.006005,0.011484,0.009463,0.018972,0.025925,0.015102,0.015440,0.016653
17598,0.418250,0.280371,0.653649,0.814706,0.601733,0.692381,1.000000,0.866687,0.420425,0.728755,...,0.021769,0.006150,0.007304,0.007851,0.009935,0.012930,0.025522,0.008336,0.010051,0.016761
17599,0.418250,0.280371,0.653649,0.814706,0.601733,0.692381,1.000000,0.866687,0.420425,0.728755,...,0.025981,0.006341,0.005336,0.002782,0.009941,0.012901,0.037105,0.006698,0.020964,0.019080


In [18]:
musicr.head()

,C,C_sharp,D,D_sharp,E,F,F_sharp,G,G_sharp,A,...,B_1sec_hist_std,B_2sec_hist_std,B_3sec_hist_std,B_4sec_hist_std,B_5sec_hist_std,B_6sec_hist_std,B_7sec_hist_std,B_8sec_hist_std,B_9sec_hist_std,B_10sec_hist_std
0,0.391866,0.271475,0.543743,0.414470,0.615711,0.905505,0.693868,1.0,0.873349,0.428765,...,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722
1,0.391006,0.270759,0.542460,0.419167,0.611465,0.895052,0.696120,1.0,0.865096,0.424725,...,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722
2,0.384290,0.269616,0.530845,0.424728,0.596871,0.879191,0.701280,1.0,0.854621,0.410783,...,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722
3,0.368410,0.266402,0.511752,0.433351,0.582881,0.878751,0.706602,1.0,0.850467,0.397394,...,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722
4,0.356740,0.260931,0.492235,0.435323,0.570510,0.877007,0.707534,1.0,0.849687,0.381047,...,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722


In [19]:
musicr.fillna(0, inplace=True)
musicr.head()

,C,C_sharp,D,D_sharp,E,F,F_sharp,G,G_sharp,A,...,B_1sec_hist_std,B_2sec_hist_std,B_3sec_hist_std,B_4sec_hist_std,B_5sec_hist_std,B_6sec_hist_std,B_7sec_hist_std,B_8sec_hist_std,B_9sec_hist_std,B_10sec_hist_std
0,0.391866,0.271475,0.543743,0.414470,0.615711,0.905505,0.693868,1.0,0.873349,0.428765,...,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722
1,0.391006,0.270759,0.542460,0.419167,0.611465,0.895052,0.696120,1.0,0.865096,0.424725,...,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722
2,0.384290,0.269616,0.530845,0.424728,0.596871,0.879191,0.701280,1.0,0.854621,0.410783,...,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722
3,0.368410,0.266402,0.511752,0.433351,0.582881,0.878751,0.706602,1.0,0.850467,0.397394,...,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722
4,0.356740,0.260931,0.492235,0.435323,0.570510,0.877007,0.707534,1.0,0.849687,0.381047,...,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722,0.002722


In [20]:
#merge the motion and music data together on time column
merged = musicr.merge(motion_all, on='millisecond', how='right')
merged.head()

,C,C_sharp,D,D_sharp,E,F,F_sharp,G,G_sharp,A,...,"(_0.4sec_hist_std_,)","(_0.6sec_hist_std_,)","(_0.8sec_hist_std_,)","(_1.0sec_hist_std_,)","(_1.2sec_hist_std_,)","(_1.4sec_hist_std_,)","(_1.6sec_hist_std_,)","(_1.8sec_hist_std_,)","(_2.0sec_hist_std_,)",segment
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.402534,0.402534,0.402534,0.402534,0.402534,0.402534,0.402534,0.402534,0.402534,silence
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.309662,0.309662,0.309662,0.309662,0.309662,0.309662,0.309662,0.309662,0.309662,silence
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.193286,0.193286,0.193286,0.193286,0.193286,0.193286,0.193286,0.193286,0.193286,silence
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.278487,0.278487,0.278487,0.278487,0.278487,0.278487,0.278487,0.278487,0.278487,silence
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.193560,0.193560,0.193560,0.193560,0.193560,0.193560,0.193560,0.193560,0.193560,silence


In [21]:
merged.fillna(0, inplace=True)
merged.head()

,C,C_sharp,D,D_sharp,E,F,F_sharp,G,G_sharp,A,...,"(_0.4sec_hist_std_,)","(_0.6sec_hist_std_,)","(_0.8sec_hist_std_,)","(_1.0sec_hist_std_,)","(_1.2sec_hist_std_,)","(_1.4sec_hist_std_,)","(_1.6sec_hist_std_,)","(_1.8sec_hist_std_,)","(_2.0sec_hist_std_,)",segment
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.402534,0.402534,0.402534,0.402534,0.402534,0.402534,0.402534,0.402534,0.402534,silence
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.309662,0.309662,0.309662,0.309662,0.309662,0.309662,0.309662,0.309662,0.309662,silence
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.193286,0.193286,0.193286,0.193286,0.193286,0.193286,0.193286,0.193286,0.193286,silence
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.278487,0.278487,0.278487,0.278487,0.278487,0.278487,0.278487,0.278487,0.278487,silence
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.193560,0.193560,0.193560,0.193560,0.193560,0.193560,0.193560,0.193560,0.193560,silence


In [22]:
merged = merged.merge(demos, on='PID', how='left')
merged

,C,C_sharp,D,D_sharp,E,F,F_sharp,G,G_sharp,A,...,Age,Sex,Height,Listen,Produce,Dance,Exercise,Tiresome,age_bin,height_bin
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,23,0.0,1.72,5,30.0,1.0,7.0,3,3,2
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,24,0.0,1.71,12,10.0,2.0,7.0,1,3,2
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,19,0.0,1.73,14,20.0,1.0,3.0,2,2,3
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,26,0.0,1.72,8,10.0,1.0,4.0,1,4,2
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,34,0.0,1.88,14,2.0,0.0,3.0,3,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517054,0.370047,0.297515,0.677903,0.787486,0.621999,0.695762,1.0,0.880282,0.364317,0.719848,...,16,1.0,1.79,40,15.0,5.0,6.0,4,1,4
517055,0.370047,0.297515,0.677903,0.787486,0.621999,0.695762,1.0,0.880282,0.364317,0.719848,...,16,1.0,1.79,40,15.0,5.0,6.0,4,1,4
517056,0.370047,0.297515,0.677903,0.787486,0.621999,0.695762,1.0,0.880282,0.364317,0.719848,...,35,0.0,1.81,1,40.0,0.5,0.5,3,5,4
517057,0.370047,0.297515,0.677903,0.787486,0.621999,0.695762,1.0,0.880282,0.364317,0.719848,...,35,0.0,1.81,1,40.0,0.5,0.5,3,5,4


In [23]:
merged_sort = merged.set_index('index').sort_index()
merged_sort

,C,C_sharp,D,D_sharp,E,F,F_sharp,G,G_sharp,A,...,Age,Sex,Height,Listen,Produce,Dance,Exercise,Tiresome,age_bin,height_bin
index,,,,,,,,,,,,,,,,,,,,,
1900-01-01 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,23,0.0,1.72,5,30.0,1.0,7.0,3,3,2
1900-01-01 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,24,0.0,1.71,12,10.0,2.0,7.0,1,3,2
1900-01-01 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,19,0.0,1.73,14,20.0,1.0,3.0,2,2,3
1900-01-01 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,26,0.0,1.72,8,10.0,1.0,4.0,1,4,2
1900-01-01 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,34,0.0,1.88,14,2.0,0.0,3.0,3,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1900-01-01 00:05:56,0.370047,0.297515,0.677903,0.787486,0.621999,0.695762,1.0,0.880282,0.364317,0.719848,...,16,1.0,1.79,40,15.0,5.0,6.0,4,1,4
1900-01-01 00:05:56,0.370047,0.297515,0.677903,0.787486,0.621999,0.695762,1.0,0.880282,0.364317,0.719848,...,16,1.0,1.79,40,15.0,5.0,6.0,4,1,4
1900-01-01 00:05:56,0.370047,0.297515,0.677903,0.787486,0.621999,0.695762,1.0,0.880282,0.364317,0.719848,...,35,0.0,1.81,1,40.0,0.5,0.5,3,5,4


In [24]:
merged_sort.isnull().sum().sort_values(ascending=False).head(10)

C                        0
G_sharp_7sec_hist_min    0
G_8sec_hist_min          0
G_9sec_hist_min          0
G_10sec_hist_min         0
G_sharp_1sec_hist_min    0
G_sharp_2sec_hist_min    0
G_sharp_3sec_hist_min    0
G_sharp_4sec_hist_min    0
G_sharp_5sec_hist_min    0
dtype: int64

In [25]:
merged_sort.to_pickle('DFs/model_data.pkl')